# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
from unityagents import UnityEnvironment

In [3]:
##!pip -q install ./python

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [4]:
import numpy as np
env = UnityEnvironment(file_name='Reacher_Windows_x86_64-20/Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [5]:
# get the default brain
brain_name = env.brain_names[0]
print(brain_name)

ReacherBrain


In [6]:
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [7]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.


In [8]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
cont = 0

In [9]:
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    cont = cont +1
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    #print("----actions-------")
    
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if(rewards[0] > 0):
        print("Actions "+ str(actions))
        print(next_states)
        print(rewards)
        print(dones)
        
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))
print (cont)

Total score (averaged over agents) this episode: 0.16849999623373152
1001


When finished, you can close the environment.

In [10]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [11]:
import random
import torch
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

In [12]:
agent = Agent(state_size, action_size, random_seed=0, num_agents = num_agents)

In [ ]:
def ddpg(n_episodes=300, print_every=100):
    scores_deque = deque(maxlen=print_every)
    scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations  
        state = states
        agent.reset()
        score = 0
        t=0
        while True:
            action = agent.act(state)
            env_info = env.step(action)[brain_name]
            reward = env_info.rewards
            next_state = env_info.vector_observations
            done = env_info.local_done
            #reward=reward2[0]
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward[0]
            t= t + 1
            if done[0]:
                break
           
        scores_deque.append(score)
        scores.append(score)
        print('\nEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if i_episode % print_every == 0:
            print('\n*Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()


Episode 1	Average Score: 0.94
Episode 2	Average Score: 1.08
Episode 3	Average Score: 0.86
Episode 4	Average Score: 0.88
Episode 5	Average Score: 1.01
Episode 6	Average Score: 1.04
Episode 7	Average Score: 1.27
Episode 8	Average Score: 1.86
Episode 9	Average Score: 2.28
Episode 10	Average Score: 3.21
Episode 11	Average Score: 4.32
Episode 12	Average Score: 5.85
Episode 13	Average Score: 6.66
Episode 14	Average Score: 8.18
Episode 15	Average Score: 9.30
Episode 16	Average Score: 10.43
Episode 17	Average Score: 11.71
Episode 18	Average Score: 13.17
Episode 19	Average Score: 14.56
Episode 20	Average Score: 15.76
Episode 21	Average Score: 16.80
Episode 22	Average Score: 17.83
Episode 23	Average Score: 18.76
Episode 24	Average Score: 19.44
Episode 25	Average Score: 20.03
Episode 26	Average Score: 20.64
Episode 27	Average Score: 21.28
Episode 28	Average Score: 21.83
Episode 29	Average Score: 22.39
Episode 30	Average Score: 22.85
Episode 31	Average Score: 23.33
Episode 32	Average Score: 23.76

In [ ]:
env.close()